In [1]:
import sentinel1decoder
import pandas as pd
import numpy as np
import logging
import math
import cmath
import struct
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy.interpolate import interp1d
import os
import pickle
from pathlib import Path


def find_dat_file(folder_directory):
    """
    Finds the dat file in the folder_directory
    """
    x = [x for x in Path(folder_directory).iterdir() if x.is_file() and x.name.endswith('.dat')]
    assert len(x) > 0, 'No dat files found.'
    x = [y for y in x if 'annot' not in y.stem]
    x = [y for y in x if 'index' not in y.stem]
    return x[0].as_posix()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Decode

In [2]:
raw_dir = '/Users/robertodelprete/Desktop/AutoFocusNet/Data/DATA/S1A_S3_RAW__0SDH_20240524T213606_20240524T213631_054018_069139_241A.SAFE'

inputfile = find_dat_file(raw_dir)
print('Input:', inputfile)
l0file = sentinel1decoder.Level0File(inputfile)

total_meta = l0file.packet_metadata
ephemeris = l0file.ephemeris

# radar_data = raw_data['echo']
last_az_idx = total_meta.index.to_flat_index()[-1][0]
tot = total_meta.shape[0]

Input: /Users/robertodelprete/Desktop/AutoFocusNet/Data/DATA/S1A_S3_RAW__0SDH_20240524T213606_20240524T213631_054018_069139_241A.SAFE/s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139.dat


In [4]:
from sentinel1decoder.utilities import read_subcommed_data

In [7]:
total_meta.keys()

Index(['Packet Version Number', 'Packet Type', 'Secondary Header Flag', 'PID',
       'PCAT', 'Sequence Flags', 'Packet Sequence Count', 'Packet Data Length',
       'Coarse Time', 'Fine Time', 'Sync', 'Data Take ID', 'ECC Number',
       'Test Mode', 'Rx Channel ID', 'Instrument Configuration ID',
       'Sub-commutated Ancilliary Data Word Index',
       'Sub-commutated Ancilliary Data Word', 'Space Packet Count',
       'PRI Count', 'Error Flag', 'BAQ Mode', 'BAQ Block Length',
       'Range Decimation', 'Rx Gain', 'Tx Ramp Rate',
       'Tx Pulse Start Frequency', 'Tx Pulse Length', 'Rank', 'PRI', 'SWST',
       'SWL', 'SAS SSB Flag', 'Polarisation', 'Temperature Compensation',
       'Calibration Mode', 'Tx Pulse Number', 'Signal Type', 'Swap Flag',
       'Swath Number', 'Number of Quads'],
      dtype='object')

In [8]:
read_subcommed_data(total_meta).iloc[0]

X-axis position ECEF           4.140284e+06
Y-axis position ECEF          -4.902956e+06
Z-axis position ECEF          -2.991130e+06
X-axis velocity ECEF           7.592896e+02
Y-axis velocity ECEF          -3.448786e+03
Z-axis velocity ECEF           6.717309e+03
POD Solution Data Timestamp    1.400622e+09
Q0 Attitude Quaternion        -3.875512e-01
Q1 Attitude Quaternion        -4.517167e-01
Q2 Attitude Quaternion         2.542285e-01
Q3 Attitude Quaternion        -7.623149e-01
Omega-X Angular Rate           4.487228e-05
Omega-Y Angular Rate          -9.061744e-04
Omega-Z Angular Rate          -5.520036e-04
Attitude Data Timestamp        1.400622e+09
Name: 0, dtype: float64

In [9]:
read_subcommed_data(total_meta)

,X-axis position ECEF,Y-axis position ECEF,Z-axis position ECEF,X-axis velocity ECEF,Y-axis velocity ECEF,Z-axis velocity ECEF,POD Solution Data Timestamp,Q0 Attitude Quaternion,Q1 Attitude Quaternion,Q2 Attitude Quaternion,Q3 Attitude Quaternion,Omega-X Angular Rate,Omega-Y Angular Rate,Omega-Z Angular Rate,Attitude Data Timestamp
0,4.140284e+06,-4.902956e+06,-2.991130e+06,759.289612,-3448.785889,6717.309082,1.400622e+09,-0.387551,-0.451717,0.254229,-0.762315,0.000045,-0.000906,-0.000552,1.400622e+09
1,4.140284e+06,-4.902956e+06,-2.991130e+06,759.289612,-3448.785889,6717.309082,1.400622e+09,-0.387551,-0.451717,0.254229,-0.762315,0.000045,-0.000906,-0.000552,1.400622e+09
2,4.140284e+06,-4.902956e+06,-2.991130e+06,759.289612,-3448.785889,6717.309082,1.400622e+09,-0.387551,-0.451717,0.254229,-0.762315,0.000045,-0.000906,-0.000552,1.400622e+09
3,4.140284e+06,-4.902956e+06,-2.991130e+06,759.289612,-3448.785889,6717.309082,1.400622e+09,-0.387551,-0.451717,0.254229,-0.762315,0.000045,-0.000906,-0.000552,1.400622e+09
4,4.140284e+06,-4.902956e+06,-2.991130e+06,759.289612,-3448.785889,6717.309082,1.400622e+09,-0.387551,-0.451717,0.254229,-0.762315,0.000045,-0.000906,-0.000552,1.400622e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,4.157662e+06,-4.987485e+06,-2.822164e+06,630.983704,-3313.088135,6799.159668,1.400622e+09,-0.389666,-0.462250,0.255058,-0.754607,0.000047,-0.000905,-0.000553,1.400622e+09
758,4.157662e+06,-4.987485e+06,-2.822164e+06,630.983704,-3313.088135,6799.159668,1.400622e+09,-0.389666,-0.462250,0.255058,-0.754607,0.000047,-0.000905,-0.000553,1.400622e+09
759,4.157662e+06,-4.987485e+06,-2.822164e+06,630.983704,-3313.088135,6799.159668,1.400622e+09,-0.389666,-0.462250,0.255058,-0.754607,0.000047,-0.000905,-0.000553,1.400622e+09
760,4.157662e+06,-4.987485e+06,-2.822164e+06,630.983704,-3313.088135,6799.159668,1.400622e+09,-0.389666,-0.462250,0.255058,-0.754607,0.000047,-0.000905,-0.000553,1.400622e+09


In [3]:
for i in range(last_az_idx):
    selected_burst = i+1
    meta = l0file.get_burst_metadata(selected_burst)
    print(i+1, meta.shape[0], (meta.shape[0]/tot*100))


1 8 0.016346212786824955
2 100 0.2043276598353119
3 50 0.10216382991765595
4 150 0.30649148975296786
5 20 0.04086553196706238
6 20 0.04086553196706238
7 60 0.12259659590118716
8 19250 39.333074518297536
9 28875 58.99961177744631
10 20 0.04086553196706238
11 20 0.04086553196706238
12 60 0.12259659590118716
13 100 0.2043276598353119
14 50 0.10216382991765595
15 150 0.30649148975296786
16 8 0.016346212786824955


In [7]:
19250 + 650

19900

In [6]:
# P8     P9
# 28875 + 19250

# SLC height: 46824

# The difference is:
28875 + 19250 - 46824


1301

In [ ]:
selected_burst = 8
meta = l0file.get_burst_metadata(selected_burst)
ephemeris = l0file.ephemeris
echo = l0file.get_burst_data(selected_burst)

raw_data =  {'metadata':meta, 'ephemeris':ephemeris, 'echo':echo}
radar_data = raw_data['echo']


In [ ]:
plt.figure(figsize=(12, 12))
plt.title("Sentinel-1 Raw I/Q Sensor Output")
plt.imshow(abs(radar_data[:,:]), origin='lower', vmin=0, vmax=15)
plt.xlabel("Fast Time (down range)")
plt.ylabel("Slow Time (cross range)")
plt.show()

## save raw data

In [ ]:
ID = 'Houston'
output = '/media/warmachine/DBDISK/SSFocus/data/Processed/' + ID + f'/raw_{Path(inputfile).stem}.pkl'
# save dictionary to person_data.pkl file
with open(output, 'wb') as fp:
    pickle.dump(raw_data, fp)
    print('dictionary saved successfully to file')

# focus raw data

In [ ]:
from pathlib import Path 

DECODED_PATH = '/Users/robertodelprete/Desktop/AutoFocusNet/Data/DECODED'
current_file_folder = DECODED_PATH / Path('s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139')
##############################
files = [x for x in Path(current_file_folder).iterdir()]
meta = [x for x in files if 'meta' in x.stem][0]
eph = [x for x in files if 'ephemeris' in x.stem][0]
echo =  [x for x in files if not ('metadata' in x.stem or 'ephemeris' in x.stem)][0]
meta = pd.read_pickle(meta)
ephemeris = pd.read_pickle(eph)
echo = pd.read_pickle(echo)
raw_data =  {'metadata':meta, 'ephemeris':ephemeris, 'echo':echo}
print(echo.shape)

In [ ]:
                            #   AZ  -   RG
radar_data = raw_data['echo'][0:1000, 0:4096]
meta = raw_data['metadata']
new_selection = meta[0:1000]
ephemeris = raw_data['ephemeris']
len_range_line = radar_data.shape[1]
len_az_line = radar_data.shape[0]

In [ ]:
# Tx pulse parameters
c = sentinel1decoder.constants.SPEED_OF_LIGHT_MPS
RGDEC = new_selection["Range Decimation"].unique()[0]
PRI = new_selection["PRI"].unique()[0]
rank = new_selection["Rank"].unique()[0]
suppressed_data_time = 320/(8*sentinel1decoder.constants.F_REF)
range_start_time = new_selection["SWST"].unique()[0] + suppressed_data_time
wavelength = sentinel1decoder.constants.TX_WAVELENGTH_M

# Sample rates
range_sample_freq = sentinel1decoder.utilities.range_dec_to_sample_rate(RGDEC)
range_sample_period = 1/range_sample_freq
az_sample_freq = 1 / PRI
az_sample_period = PRI

# Fast time vector - defines the time axis along the fast time direction (azimuth)
sample_num_along_range_line = np.arange(0, len_range_line, 1)
fast_time_vec = range_start_time + (range_sample_period * sample_num_along_range_line)

# Slant range vector - defines R0, the range of closest approach, for each range cell
slant_range_vec = ((rank * PRI) + fast_time_vec) * c/2
    
# Axes - defines the frequency axes in each direction after FFT
SWL = len_range_line/range_sample_freq

az_freq_vals = np.arange(-az_sample_freq/2, az_sample_freq/2, 1/(PRI*len_az_line))
range_freq_vals = np.arange(-range_sample_freq/2, range_sample_freq/2, 1/SWL)

# Spacecraft velocity - numerical calculation of the effective spacecraft velocity
ecef_vels = ephemeris.apply(lambda x: math.sqrt(x["X-axis velocity ECEF"]**2 + x["Y-axis velocity ECEF"]**2 +x["Z-axis velocity ECEF"]**2), axis=1)
velocity_interp = interp1d(ephemeris["POD Solution Data Timestamp"].unique(), ecef_vels.unique(), fill_value="extrapolate")
x_interp = interp1d(ephemeris["POD Solution Data Timestamp"].unique(), ephemeris["X-axis position ECEF"].unique(), fill_value="extrapolate")
y_interp = interp1d(ephemeris["POD Solution Data Timestamp"].unique(), ephemeris["Y-axis position ECEF"].unique(), fill_value="extrapolate")
z_interp = interp1d(ephemeris["POD Solution Data Timestamp"].unique(), ephemeris["Z-axis position ECEF"].unique(), fill_value="extrapolate")
space_velocities = new_selection.apply(lambda x: velocity_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)

# D computation
x_positions = new_selection.apply(lambda x: x_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)
y_positions = new_selection.apply(lambda x: y_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)
z_positions = new_selection.apply(lambda x: z_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)

position_array = np.transpose(np.vstack((x_positions, y_positions, z_positions)))

a = sentinel1decoder.constants.WGS84_SEMI_MAJOR_AXIS_M
b = sentinel1decoder.constants.WGS84_SEMI_MINOR_AXIS_M
H = np.linalg.norm(position_array, axis=1)
W = np.divide(space_velocities, H)
lat = np.arctan(np.divide(position_array[:, 2], position_array[:, 0]))
local_earth_rad = np.sqrt(
    np.divide(
        (np.square(a**2 * np.cos(lat)) + np.square(b**2 * np.sin(lat))),
        (np.square(a * np.cos(lat)) + np.square(b * np.sin(lat)))
    )
)
cos_beta = (np.divide(np.square(local_earth_rad) + np.square(H) - np.square(slant_range_vec[:, np.newaxis]) , 2 * local_earth_rad * H))
ground_velocities = local_earth_rad * W * cos_beta

effective_velocities = np.sqrt(space_velocities * ground_velocities)

D = np.sqrt(
    1 - np.divide(
        wavelength**2 * np.square(az_freq_vals),
        4 * np.square(effective_velocities)
    )
).T

# We're only interested in keeping D, so free up some memory by deleting these large arrays.
# del effective_velocities
del ground_velocities
del cos_beta
del local_earth_rad
del H
del W
del lat

In [ ]:
ecef_vels = ephemeris.apply(lambda x: math.sqrt(x["X-axis velocity ECEF"]**2 + x["Y-axis velocity ECEF"]**2 +x["Z-axis velocity ECEF"]**2), axis=1)
ecef_vels[0]

In [ ]:
plt.plot(effective_velocities[:,0])
plt.plot(effective_velocities[0,:])

In [ ]:
# FFT each range line
radar_data = np.fft.fft(radar_data, axis=1)
# FFT each azimuth line
radar_data = np.fft.fftshift(np.fft.fft(radar_data, axis=0), axes=0)

In [ ]:
TXPSF = new_selection["Tx Pulse Start Frequency"].unique()[0]
TXPRR = new_selection["Tx Ramp Rate"].unique()[0]
TXPL = new_selection["Tx Pulse Length"].unique()[0]

num_tx_vals = int(TXPL*range_sample_freq)
tx_replica_time_vals = np.linspace(-TXPL/2, TXPL/2, num=num_tx_vals)
phi1 = TXPSF + TXPRR*TXPL/2
phi2 = TXPRR/2
tx_replica = np.exp(2j * np.pi * (phi1*tx_replica_time_vals + phi2*tx_replica_time_vals**2))

# Create range filter from replica pulse
range_filter = np.zeros(len_range_line, dtype=complex)
index_start = np.ceil((len_range_line-num_tx_vals)/2)-1
index_end = num_tx_vals+np.ceil((len_range_line-num_tx_vals)/2)-2

range_filter[int(index_start):int(index_end+1)] = tx_replica
range_filter = np.conjugate(np.fft.fft(range_filter))

In [ ]:
radar_data = raw_data['echo']
meta = raw_data['metadata']
ephemeris = raw_data['ephemeris']

len_range_line = radar_data.shape[1]
len_az_line = radar_data.shape[0]

# Tx pulse parameters
c = sentinel1decoder.constants.SPEED_OF_LIGHT_MPS
RGDEC = meta["Range Decimation"].unique()[0]
PRI = meta["PRI"].unique()[0]
rank = meta["Rank"].unique()[0]
suppressed_data_time = 320/(8*sentinel1decoder.constants.F_REF)
range_start_time = meta["SWST"].unique()[0] + suppressed_data_time
wavelength = sentinel1decoder.constants.TX_WAVELENGTH_M

# Sample rates
range_sample_freq = sentinel1decoder.utilities.range_dec_to_sample_rate(RGDEC)
range_sample_period = 1/range_sample_freq
az_sample_freq = 1 / PRI
az_sample_period = PRI

# Fast time vector - defines the time axis along the fast time direction
sample_num_along_range_line = np.arange(0, len_range_line, 1)
fast_time_vec = range_start_time + (range_sample_period * sample_num_along_range_line)

# Slant range vector - defines R0, the range of closest approach, for each range cell
slant_range_vec = ((rank * PRI) + fast_time_vec) * c/2
    
# Axes - defines the frequency axes in each direction after FFT
SWL = len_range_line/range_sample_freq
az_freq_vals = np.arange(-az_sample_freq/2, az_sample_freq/2, 1/(PRI*len_az_line))
range_freq_vals = np.arange(-range_sample_freq/2, range_sample_freq/2, 1/SWL)

# Spacecraft velocity - numerical calculation of the effective spacecraft velocity
ecef_vels = ephemeris.apply(lambda x: math.sqrt(x["X-axis velocity ECEF"]**2 + x["Y-axis velocity ECEF"]**2 +x["Z-axis velocity ECEF"]**2), axis=1)
velocity_interp = interp1d(ephemeris["POD Solution Data Timestamp"].unique(), ecef_vels.unique(), fill_value="extrapolate")
x_interp = interp1d(ephemeris["POD Solution Data Timestamp"].unique(), ephemeris["X-axis position ECEF"].unique(), fill_value="extrapolate")
y_interp = interp1d(ephemeris["POD Solution Data Timestamp"].unique(), ephemeris["Y-axis position ECEF"].unique(), fill_value="extrapolate")
z_interp = interp1d(ephemeris["POD Solution Data Timestamp"].unique(), ephemeris["Z-axis position ECEF"].unique(), fill_value="extrapolate")
space_velocities = meta.apply(lambda x: velocity_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)

x_positions = meta.apply(lambda x: x_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)
y_positions = meta.apply(lambda x: y_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)
z_positions = meta.apply(lambda x: z_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)

position_array = np.transpose(np.vstack((x_positions, y_positions, z_positions)))

a = sentinel1decoder.constants.WGS84_SEMI_MAJOR_AXIS_M
b = sentinel1decoder.constants.WGS84_SEMI_MINOR_AXIS_M
H = np.linalg.norm(position_array, axis=1)
W = np.divide(space_velocities, H)
lat = np.arctan(np.divide(position_array[:, 2], position_array[:, 0]))
local_earth_rad = np.sqrt(
    np.divide(
        (np.square(a**2 * np.cos(lat)) + np.square(b**2 * np.sin(lat))),
        (np.square(a * np.cos(lat)) + np.square(b * np.sin(lat)))
    )
)
cos_beta = (np.divide(np.square(local_earth_rad) + np.square(H) - np.square(slant_range_vec[:, np.newaxis]) , 2 * local_earth_rad * H))
ground_velocities = local_earth_rad * W * cos_beta

effective_velocities = np.sqrt(space_velocities * ground_velocities)

D = np.sqrt(
    1 - np.divide(
        wavelength**2 * np.square(az_freq_vals),
        4 * np.square(effective_velocities)
    )
).T

# We're only interested in keeping D, so free up some memory by deleting these large arrays.
del effective_velocities
del ground_velocities
del cos_beta
del local_earth_rad
del H
del W
del lat
# FFT each range line
radar_data = np.fft.fft(radar_data, axis=1)

# FFT each azimuth line
radar_data = np.fft.fftshift(np.fft.fft(radar_data, axis=0), axes=0)
# Create replica pulse
TXPSF = meta["Tx Pulse Start Frequency"].unique()[0]
TXPRR = meta["Tx Ramp Rate"].unique()[0]
TXPL = meta["Tx Pulse Length"].unique()[0]
num_tx_vals = int(TXPL*range_sample_freq)
tx_replica_time_vals = np.linspace(-TXPL/2, TXPL/2, num=num_tx_vals)
phi1 = TXPSF + TXPRR*TXPL/2
phi2 = TXPRR/2
tx_replica = np.exp(2j * np.pi * (phi1*tx_replica_time_vals + phi2*tx_replica_time_vals**2))

# Create range filter from replica pulse
range_filter = np.zeros(len_range_line, dtype=complex)
index_start = np.ceil((len_range_line-num_tx_vals)/2)-1
index_end = num_tx_vals+np.ceil((len_range_line-num_tx_vals)/2)-2
range_filter[int(index_start):int(index_end+1)] = tx_replica
range_filter = np.conjugate(np.fft.fft(range_filter))

# Apply filter
radar_data = np.multiply(radar_data, range_filter)

del range_filter
del tx_replica

# Create RCMC filter
range_freq_vals = np.linspace(-range_sample_freq/2, range_sample_freq/2, num=len_range_line)
rcmc_shift = slant_range_vec[0] * (np.divide(1, D) - 1)
rcmc_filter = np.exp(4j * np.pi * range_freq_vals * rcmc_shift / c)

# Apply filter
radar_data = np.multiply(radar_data, rcmc_filter)

del rcmc_shift
del rcmc_filter
del range_freq_vals
radar_data = np.fft.ifftshift(np.fft.ifft(radar_data, axis=1), axes=1)
# Create filter
az_filter = np.exp(4j * np.pi * slant_range_vec * D / wavelength)

# Apply filter
radar_data = np.multiply(radar_data, az_filter)

del az_filter

radar_data = np.fft.ifft(radar_data, axis=0)

In [ ]:
focused = focus(raw_data)

In [ ]:
output = '/Users/robertodelprete/Desktop/AutoFocusNet/Data/DECODED/s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139/focused-s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139.pkl'
with open(output, 'wb') as fp:
    pickle.dump(focused, fp)
    print('dictionary saved successfully to file')

## Plot raw data

In [ ]:
plt.figure(figsize=(16, 16))
plt.title("Sentinel-1 Raw I/Q Sensor Output")
raw_data_mean = np.mean(abs(focused))
raw_data_std = np.std(abs(focused))
raw_data_max = np.max(abs(focused))
plt.imshow(abs(focused[:,:]), origin='lower', vmin=raw_data_mean-raw_data_std*0.5, vmax=raw_data_mean+raw_data_std*3)
plt.xlabel("Fast Time (down range)")
plt.ylabel("Slow Time (cross range)")
plt.show()